In [1]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import networkx as nx
%matplotlib inline

In [2]:
df = pd.read_csv('DSL-StrongPasswordData.csv')
df.head()

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,s002,1,1,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,...,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742
1,s002,1,2,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,...,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747
2,s002,1,3,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,...,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945
3,s002,1,4,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,...,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813
4,s002,1,5,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,...,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818


In [4]:
subjects = df['subject'].unique()

In [5]:
subjects_to_int = {subject: i  for i, subject in enumerate(subjects)}
int_to_subjects = {i: subject for i, subject in enumerate(subjects)}

In [6]:
df = df.replace(subjects_to_int)

C:\Users\skhim\AppData\Local\Temp\ipykernel_10072\1725018810.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(subjects_to_int)


In [7]:
df.head()

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,0,1,1,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,...,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742
1,0,1,2,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,...,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747
2,0,1,3,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,...,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945
3,0,1,4,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,...,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813
4,0,1,5,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,...,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818


In [8]:
data_raw = df.values
print(data_raw)
data_raw.shape

[[ 0.      1.      1.     ...  0.3509  0.2171  0.0742]
 [ 0.      1.      2.     ...  0.2756  0.1917  0.0747]
 [ 0.      1.      3.     ...  0.2847  0.1762  0.0945]
 ...
 [50.      8.     48.     ...  0.2017  0.0983  0.0905]
 [50.      8.     49.     ...  0.1917  0.0938  0.0931]
 [50.      8.     50.     ...  0.1993  0.1186  0.1018]]


(20400, 34)

In [9]:
data=data_raw[ :, 3:-1]
labels_raw = df['subject'].values
labels= labels_raw.reshape(labels_raw.shape[0],1)
data=np.hstack([data, labels])
print(data)

[[1.491e-01 3.979e-01 2.488e-01 ... 3.509e-01 2.171e-01 0.000e+00]
 [1.111e-01 3.451e-01 2.340e-01 ... 2.756e-01 1.917e-01 0.000e+00]
 [1.328e-01 2.072e-01 7.440e-02 ... 2.847e-01 1.762e-01 0.000e+00]
 ...
 [9.390e-02 1.189e-01 2.500e-02 ... 2.017e-01 9.830e-02 5.000e+01]
 [9.230e-02 1.294e-01 3.710e-02 ... 1.917e-01 9.380e-02 5.000e+01]
 [5.960e-02 1.310e-01 7.140e-02 ... 1.993e-01 1.186e-01 5.000e+01]]


In [10]:
np.random.shuffle(data)
x=data[ : , :-1]
y=data[:,-1]
x.shape

(20400, 30)

In [11]:
scaler = MinMaxScaler()
x = scaler.fit_transform(x)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)

In [14]:
Y_train = to_categorical(y_train, num_classes=51)
Y_test=to_categorical(y_test,num_classes=51)


In [15]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_train.shape

(18360, 30, 1)

In [16]:
from tensorflow.keras.models import load_model

# Load the entire model from the file
rnn = load_model('LSTM.h5')
# rnn = load_model('')

ValueError: Unrecognized keyword arguments: ['batch_shape']

In [ ]:
print(X_test.shape)
predictions_prob= rnn.predict(X_test)
y_pred=[np.argmax(i) for i in predictions_prob]
print(y_pred)
print(y_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
print("Accuracy: ",accuracy)
print("Precison: ",precision)
print("Recall: ",recall)
print("F1 Score: ",f1)

In [ ]:
plot_model(rnn, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
plot_model(rnn, to_file='model_plot.png', show_shapes=True, show_layer_names=True)